In [6]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0867, -0.0497,  0.0236,  0.0235, -0.0177, -0.4094,  0.0358, -0.0416,
          0.0979,  0.1360],
        [-0.0884, -0.1962, -0.1121,  0.2121, -0.0376, -0.1609, -0.0491, -0.0103,
          0.0900,  0.1890]], grad_fn=<AddmmBackward0>)

In [9]:
class MLP(nn.Module):
    # 用模型参数声明层这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的弗雷Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层
        
    # 定义模型的前向传播。即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [10]:
net = MLP()
net(X)

tensor([[-0.2329,  0.0289,  0.1264,  0.0372,  0.0281, -0.1115, -0.2235,  0.0363,
          0.0263, -0.0048],
        [-0.2677,  0.0132,  0.0488,  0.0884,  0.0576,  0.0786, -0.1774,  0.0755,
          0.0089, -0.0986]], grad_fn=<AddmmBackward0>)

In [11]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里， module是Module子类的一个实例，我们把它保存在‘Module’类的成员
            # 变量_modules中 module的类型是OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [12]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0414,  0.0511,  0.4149,  0.0373, -0.0695, -0.2275,  0.0234,  0.3457,
          0.2817,  0.0039],
        [-0.0059,  0.0124,  0.4232,  0.1419, -0.0251, -0.2316,  0.0899,  0.2469,
          0.1544, -0.0935]], grad_fn=<AddmmBackward0>)

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2096, grad_fn=<SumBackward0>)

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1882, grad_fn=<SumBackward0>)